In [20]:
!jupyter nbconvert --to script DistributedDataParallel_script.ipynb
#!python DistributedDataParallel_script.py --issingle
!rm -rf MNIST_data

[NbConvertApp] Converting notebook DistributedDataParallel_script.ipynb to script
[NbConvertApp] Writing 11542 bytes to DistributedDataParallel_script.py


## Initialize Azure ML

In [21]:
from azureml.core import Workspace, Experiment, Run
ws = Workspace(subscription_id='42ae47bd-b19b-42c1-b0b9-19fd5be9d51b',
              resource_group='bert-base',
              workspace_name='SubstrateIntelligenceNLR-WS2')

experiment = Experiment(ws, 'MNIST-singlegpu')

from azureml.core import ComputeTarget
compute_target = ComputeTarget(ws, 'nc24sv3')

### Download and upload data 

In [22]:
from  DistributedDataParallel_script import loaddata

datadir = '../MNIST_data'
blobpath = 'MNIST_data/'
loaddata(datadir)

ds = ws.get_default_datastore()


ds.upload(datadir, target_path=blobpath)

loading data to ../MNIST_data
Dataset MNIST
    Number of datapoints: 60000
    Root location: ../MNIST_data
    Split: Train
60000 10000
(784,) 5
Uploading an estimated of 6 files
Target already exists. Skipping upload for MNIST_data/MNIST/raw/train-images-idx3-ubyte
Target already exists. Skipping upload for MNIST_data/MNIST/raw/t10k-images-idx3-ubyte
Target already exists. Skipping upload for MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte
Target already exists. Skipping upload for MNIST_data/MNIST/raw/train-labels-idx1-ubyte
Target already exists. Skipping upload for MNIST_data/MNIST/processed/test.pt
Target already exists. Skipping upload for MNIST_data/MNIST/processed/training.pt
Uploaded 0 files


$AZUREML_DATAREFERENCE_122f661fb50c4b948b286bd8d8cfb8a1

### Single GPU

In [23]:
from azureml.train.dnn import PyTorch
estimator = PyTorch(source_directory = '.',
                    pip_packages = ['tqdm'],
                    compute_target = compute_target,
                    entry_script = 'DistributedDataParallel_script.py',
                    script_params = {
                       '--issingle':"",
                       '--datapath':ds.path(blobpath).as_mount()
                    },
                    use_gpu=True,
                   )

run = experiment.submit(estimator)

WARNING - framework_version is not specified, defaulting to version 1.1.


In [24]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

### Distributed data parallel AML job - Not working
2 nodes 4 GPUS
Follow this
https://github.com/microsoft/AzureML-BERT/blob/master/pretrain/PyTorch/azureml_adapter.py

In [25]:
from azureml.train.dnn import PyTorch
from azureml.core.runconfig import MpiConfiguration

mpi = MpiConfiguration()
mpi.process_count_per_node = 4

estimator_distrib = PyTorch(source_directory = '.',
                    pip_packages = ['tqdm'],
                   compute_target = compute_target,
                   entry_script = 'DistributedDataParallel_script.py',
                   script_params = {'--isaml':"", '--backend' : 'nccl', '--datapath':ds.path(blobpath).as_mount()},
                   use_gpu=True,
                   distributed_training = mpi,
                    node_count = 2
                   )


WARNING - framework_version is not specified, defaulting to version 1.1.


In [26]:
run_distrib = experiment.submit(estimator_distrib)
RunDetails(run_distrib).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…